### Натуральный ряд чисел


---

Условие задачи:

* Весь ряд натуральных чисел по возрастанию записан без пробелов в строку. Строка пронумерована начиная с 0.

---

Требуется написать функцию:
1. На вход должно подаваться число N - индекс цифры в строке.
2. Функция должна возвращать цифру, которая находится под индексом N в строке.

---

Пример:
* Строка: 1234567891011121314151617181920212223
* N = 14
* f(N) = 2

---


### Моя первая версия алгоритма


* начальная настройка для больших чисел


In [ ]:
import sys

sys.set_int_max_str_digits(0)

* реализация алгоритма


In [ ]:
def get_current_capacity_digits(capacity: int) -> int:
    """
    Получение числа цифр, которые есть в числах текущей разрядности.

    Механизм прост: всего чисел в разрядности 9 * 10**n
    Например: 9, 90, 900, и т.п.
    n = capacity - 1, легко проверяется.

    Остается посчитать кол-во цифр: кол-во числе * capacity
    """
    return 9 * capacity * (10 ** (capacity - 1))


def get_digit(N: int) -> int:
    """
    Поиск по строке, состоящей из всех натуральных чисел.
    Считается, что первый индекс в строке - 0.
    Возращает цифру под индексом N в этой строке.
    """
    # capacity - Искомая разрядность числа.
    # Внутри данного числа и находится искомая цифра.
    capacity = 1

    # not_less_capacity_digits - счётчик числа цифр,
    # которые лежат в числах той же или большей разрядности.
    # По умолчанию разрядность 1 => все цифры (N) лежат в number.
    not_less_capacity_digits = N

    # current_capacity_digits - счётчик числа цифр,
    # которые лежат в числах той же разрядности.
    current_capacity_digits = get_current_capacity_digits(capacity)

    # Цикл для поиска разряда числа, внутри которого лежит цифра.
    # number >= current_capacity_digits, значит оно точно
    # большей разрядности.
    while not_less_capacity_digits >= current_capacity_digits:
        not_less_capacity_digits -= current_capacity_digits

        capacity += 1
        current_capacity_digits = get_current_capacity_digits(
            capacity
        )

    # numbers_up_to - все числа, которые шли до числа,
    # в котором содержится искомая цифра.
    # Всего цифр в этих числах = not_less_capacity_digits,
    # Разрядность этих чисел = capacity,
    # Следовательно их кол-во = кол-во цифр // разрядность.
    numbers_up_to = not_less_capacity_digits // capacity

    # searched_digit_index - индекс искомой цифры в её числе.
    searched_digit_index = not_less_capacity_digits % capacity

    # Получение цифры из знания о её индексе.
    # power - степень числа,
    # на которое необходимо разделить numbers_up_to,
    # чтобы последняя цифра результирующего числа была равна искомой.
    power = capacity - searched_digit_index - 1

    if searched_digit_index == 0:
        result_digit = (numbers_up_to // (10**power)) + 1

    else:
        result_digit = (numbers_up_to // (10**power)) % 10

    # # ! ИНФОРМАЦИЯ ДЛЯ ОТЛАДКИ
    # # ! Не используется в основной функции.
    # print(
    #     "Searched digit: {}\nCapacity: {}\nNumbers up to: {}\n"
    #     "Searched digit local index: {}".format(
    #         N,
    #         capacity,
    #         numbers_up_to,
    #         searched_digit_index,
    #     )
    # )

    # Возвращаем искомую цифру.
    return result_digit

* тестирование алгоритма


In [ ]:
print("Result: ", get_digit(eval(input())))

### Эталонный алгоритм для Python


* начальная настройка для больших чисел


In [ ]:
import sys

sys.set_int_max_str_digits(0)

#### Оригинальный вариант


* автор оригинального алгоритма: @tgfromrus


* реализация алгоритма


In [ ]:
def fast_get_digit(n):
    digits = 1
    count = 9
    start = 1

    while n > digits * count:
        n -= digits * count
        digits += 1
        count *= 10
        start *= 10

    number = start + (n - 1) // digits
    index = (n - 1) % digits

    return str(number)[index]

* тестирование алгоритма


In [ ]:
print("Result: ", fast_get_digit(eval(input())))

* Время счёта 10^500_000:
* ~46.1s
* < 47s


#### Моя оптимизация алгоритма


* начальная настройка для больших чисел


In [ ]:
def fast_get_digit(n):
    digits = 1
    count = 9

    while n > digits * count:
        n -= digits * count
        digits += 1
        count *= 10

    number = (count // 9) + (n - 1) // digits
    index = (n - 1) % digits

    return str(number)[index]

* тестирование алгоритма


In [ ]:
print("Result: ", fast_get_digit(10**500_000))

---

* Время счёта 10^500_000:
* ~39.5s
* < 40s

---

* Время счёта 10^1_000_000:
* ~156.27s

---


### Эталонная реализация на Rust


* реализация алгоритма


In [ ]:
use num_bigint::BigUint;
use num_traits::cast::ToPrimitive;

fn main() {
    // n = 2_500_000
    let mut n = BigUint::new(vec![10]).pow(100_000);
    n *= BigUint::new(vec![25])

    let mut digits = BigUint::new(vec![1]);
    let mut count = BigUint::new(vec![9]);

    let one = BigUint::new(vec![1]);
    let nine = BigUint::new(vec![9]);
    let ten = BigUint::new(vec![10]);

    while n > &digits * &count {
        n -= &digits * &count;
        digits += &one;
        count *= &ten;
    }

    let number = (&count / &nine) + (&n - &one) / &digits;
    let index = ((n - one) % digits).to_usize().expect("Lol, usize error");

    let result = number
        .to_string()
        .chars()
        .nth(index)
        .expect("Lol, result error");

    println!("{}", result);
}


* финальное тестирование алгоритма


n            |Цифра под индексом 10^n|Время выполнения, с|Время выполнения, м|
-------------|-----------------------|-------------------|-------------------|
100_000      |  1                    |  0.57             |  ~0.00            |
500_000      |  2                    |  14.53            |  ~0.00            |
1_000_000    |  0                    |  60.37            |   1.00            |
1_500_000    |  4                    |  133.45           |   2.23            |
2_500_000    |  3                    |  377.31           |   6.29            |
5_000_000    |  9                    |  1580.68          |   26.34           |
